In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# import os
# os.chdir('/content/drive/My Drive/translate')

In [ ]:
drive.flush_and_unmount()

In [ ]:
!sudo apt install rar

In [ ]:
!rar x datasets.rar

In [ ]:
!pip install transformers datasets evaluate sacrebleu nltk keras_nlp
!pip install sentencepiece

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AdamWeightDecay
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer
import datasets
import tensorflow as tf
# import keras_nlp
# from nltk.translate.bleu_score import sentence_bleu
# import evaluate
import numpy as np
import time

In [ ]:
split_char = '⫯'
data_name = 'PTT_2023_08_06'
model_name = f'mT5_small_PTT_2023_08_06_Thu_Aug_17_081645_2023.ckpt'
# model_name = f'mT5_small_{data_name}_{time.ctime().replace(" ", "_").replace(":", "")}.ckpt'
checkpoint = "google/mt5-small"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "translate engTyping to Traditional Chinese:".split()

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
model.load_weights('models/mT5_small_PTT_2023_08_06_Thu_Aug_17_081645_2023.ckpt')

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
tf_train_set = tf.data.Dataset.load(f'datasets/mT5_{data_name}_tf_train_dataset.tfrecord')
tf_val_set = tf.data.Dataset.load(f'datasets/mT5_{data_name}_tf_val_dataset.tfrecord')

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.compile(optimizer=optimizer, metrics=["accuracy"])  # No loss argument!

# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_set)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(f'models/{model_name}',save_best_only=True, save_weights_only=True),
    tf.keras.callbacks.TensorBoard(log_dir=f'logs/{model_name}_logs')
]

model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=1, callbacks=callbacks)
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/translate')
model.save_pretrained(f'models/{model_name}')

In [ ]:
split_char = '⫯'
punctuations = '、，。？！：；'

def engTyping_insert_split_char(sentence: str, split_char: str) -> str:
    insert_times = 0
    sentence_list = list(sentence)
    for i, char in enumerate(sentence):
        if char in ' 6347' + punctuations:
            sentence_list.insert(i + insert_times + 1, split_char)
            insert_times += 1
    return ''.join(sentence_list[:-1])

In [ ]:
inputs = tokenizer(prefix + engTyping_insert_split_char(input('?:'), split_char).split(split_char), is_split_into_words=True, return_tensors="tf", truncation=True).input_ids
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))